## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import json
import requests
import pandas as pd
from datetime import datetime, date

## Download

In [6]:
POWER_BI_QUERY_URL = (
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata"
)
REQUEST_PARAMS = (("synchronous", "true"),)
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"r","Entity":"Region_Zip_Count","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Region"},"Name":"Region_Zip_Count.Region"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Case_Count"}},"Function":0},"Name":"Sum(Region_Zip_Count.Case_Count)"},{"Arithmetic":{"Left":{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Case_Count"}},"Function":0}},"Right":{"ScopedEval":{"Expression":{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Case_Count"}},"Function":0}},"Scope":[]}},"Operator":3},"Name":"Divide(Sum(Region_Zip_Count.Case_Count), ScopedEval(Sum(Region_Zip_Count.Case_Count), []))"},{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Zip_Code"},"Name":"Region_Zip_Count.Zip_Code"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Change_Previous_Date"}},"Function":0},"Name":"Sum(Region_Zip_Count.Change_Previous_Date)"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Recovered_Count"}},"Function":0},"Name":"Sum(Region_Zip_Count.Recovered_Count)"},{"Measure":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Recovered_Rate"},"Name":"Region_Zip_Count.Recovered_Rate"},{"Measure":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Recovered_Rate_Change"},"Name":"Region_Zip_Count.Recovered_Rate_Change"}],"OrderBy":[{"Direction":1,"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"r"}},"Property":"Region"}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0]},{"Projections":[1,2,3,4,5,6,7],"Subtotal":2}]},"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"Version":1}}}]},"CacheKey":"{\\"Commands\\":[{\\"SemanticQueryDataShapeCommand\\":{\\"Query\\":{\\"Version\\":2,\\"From\\":[{\\"Name\\":\\"r\\",\\"Entity\\":\\"Region_Zip_Count\\",\\"Type\\":0}],\\"Select\\":[{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Region\\"},\\"Name\\":\\"Region_Zip_Count.Region\\"},{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Case_Count\\"}},\\"Function\\":0},\\"Name\\":\\"Sum(Region_Zip_Count.Case_Count)\\"},{\\"Arithmetic\\":{\\"Left\\":{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Case_Count\\"}},\\"Function\\":0}},\\"Right\\":{\\"ScopedEval\\":{\\"Expression\\":{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Case_Count\\"}},\\"Function\\":0}},\\"Scope\\":[]}},\\"Operator\\":3},\\"Name\\":\\"Divide(Sum(Region_Zip_Count.Case_Count), ScopedEval(Sum(Region_Zip_Count.Case_Count), []))\\"},{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Zip_Code\\"},\\"Name\\":\\"Region_Zip_Count.Zip_Code\\"},{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Change_Previous_Date\\"}},\\"Function\\":0},\\"Name\\":\\"Sum(Region_Zip_Count.Change_Previous_Date)\\"},{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Recovered_Count\\"}},\\"Function\\":0},\\"Name\\":\\"Sum(Region_Zip_Count.Recovered_Count)\\"},{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Recovered_Rate\\"},\\"Name\\":\\"Region_Zip_Count.Recovered_Rate\\"},{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Recovered_Rate_Change\\"},\\"Name\\":\\"Region_Zip_Count.Recovered_Rate_Change\\"}],\\"OrderBy\\":[{\\"Direction\\":1,\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"r\\"}},\\"Property\\":\\"Region\\"}}}]},\\"Binding\\":{\\"Primary\\":{\\"Groupings\\":[{\\"Projections\\":[0]},{\\"Projections\\":[1,2,3,4,5,6,7],\\"Subtotal\\":2}]},\\"DataReduction\\":{\\"DataVolume\\":3,\\"Primary\\":{\\"Window\\":{\\"Count\\":500}}},\\"Version\\":1}}}]}","QueryId":"","ApplicationContext":{"DatasetId":"f4cfdb16-2900-473c-912e-f4ae9e29a467","Sources":[{"ReportId":"132ae928-07b3-4efa-865e-9c8c7f4602e4","VisualId":"85a3d3c805c6d81321ea"}]}}],"cancelQueries":[],"modelId":282655}'

In [7]:
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,da;q=0.8",
    "ActivityId": "eb83be65-f925-1b74-eab2-c44e42f84faa",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Content-Length": "4309",
    "Content-Type": "application/json;charset=UTF-8",
    "DNT": "1",
    "Host": "wabi-us-gov-iowa-api.analysis.usgovcloudapi.net",
    "Origin": "https://app.powerbigov.us",
    "Pragma": "no-cache",
    "Referer": "https://app.powerbigov.us/",
    "RequestId": "51f1f9e4-8633-2419-778d-a7668ee8b56e",
    # "sec-ch-ua": " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
    # "sec-ch-ua-mobile": ?0
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
    "X-PowerBI-ResourceKey": "fca0e553-4f29-4989-a8f0-aa88080c3c8a",
}
json_data = json.loads(REQUEST_BODY)

In [8]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/Users/pdo/.local/share/virtualenvs/california-coronavirus-scrapers-l_auzzjX/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
data = response.json()

## Parse

In [10]:
timestamp = response.headers["Date"]

In [11]:
ds = data["results"][0]["result"]["data"]["dsr"]["DS"][0]

In [12]:
zip_codes = ds["ValueDicts"]["D0"]

In [13]:
rows = ds["PH"][0]["DM0"]

In [14]:
confirmed_cases = []

cases = 0

for row in rows:
    # in some cases, data are encoded in separate keys

    # pull out grouping and the list containing values
    grouping = row["G0"]
    cells = row["M"][0]["DM1"]

    # loop through the list of values
    for cell in cells:
        keys = cell.keys()
        index = cell["C"][0]

        # if the list of values is one, that means it contains identical numbers to the row previous
        if len(cell["C"]) == 1:
            pass
        else:
            cases = cell["C"][1]
        confirmed_cases.append(
            (
                index,
                cases,
                grouping,
            )
        )

In [15]:
matched = [
    (grouping, zip_codes[index], cases) for index, cases, grouping in confirmed_cases
]

In [16]:
df = pd.DataFrame(matched, columns=["area", "zip_code", "confirmed_cases"])

Match up place names with zip codes

In [17]:
df = df.loc[df.zip_code != "Other"]

In [18]:
zip_code_names = {
    "93902": "Prunedale",
    "93907": "Salinas",
    "95004": "Aromas",
    "95012": "Castroville",
    "95039": "Moss Landing",
    "95076": "Watsonville",
    "93920": "Big Sur",
    "93921": "Carmel",
    "93923": "Carmel",
    "93924": "Carmel Valley",
    "93933": "Marina",
    "93940": "Monterey",
    "93942": "Monterey",
    "93950": "Pacific Grove",
    "93953": "Pebble Beach",
    "93955": "Seaside",
    "93901": "Salinas",
    "93905": "Salinas",
    "93906": "Salinas",
    "93908": "Salinas",
    "93912": "Boronda",
    "93915": "Salinas",
    "93962": "Spreckels",
    "93450": "San Ardo",
    "93925": "Chualar",
    "93926": "Gonzales",
    "93927": "Greenfield",
    "93930": "King City",
    "93954": "San Lucas",
    "93960": "Soledad",
    "93922": "Carmel",
    "93426": "Bradley",
    "93451": "San Miguel",
}

In [19]:
df["area_name"] = df.zip_code.map(zip_code_names)

In [20]:
df["area_name"] = df["zip_code"] + ": " + df["area_name"]

In [21]:
df = df[df.zip_code != "Unknown"]

Set column with county date

In [22]:
df["county_date"] = pd.to_datetime(timestamp)

In [23]:
df["county_date"] = df["county_date"].dt.tz_convert("US/Pacific")

In [24]:
df["county_date"] = df["county_date"].dt.strftime("%Y-%m-%d")

In [25]:
df.insert(0, "county", "Monterey")

Clean up for export

In [26]:
export_df = df[
    ["county", "area_name", "confirmed_cases", "county_date", "zip_code"]
].rename(columns={"area_name": "area", "zip_code": "zip"})

In [37]:
export_df

,county,area,confirmed_cases,county_date,zip
0,Monterey,93902: Prunedale,68,2021-10-21,93902
1,Monterey,93907: Salinas,1993,2021-10-21,93907
2,Monterey,95004: Aromas,145,2021-10-21,95004
3,Monterey,95012: Castroville,1595,2021-10-21,95012
4,Monterey,95039: Moss Landing,145,2021-10-21,95039
5,Monterey,95076: Watsonville,1447,2021-10-21,95076
6,Monterey,93920: Big Sur,61,2021-10-21,93920
7,Monterey,93921: Carmel,58,2021-10-21,93921
8,Monterey,93922: Carmel,31,2021-10-21,93922
9,Monterey,93923: Carmel,321,2021-10-21,93923


## Vet

In [27]:
try:
    assert not len(export_df) < 33
except AssertionError:
    raise AssertionError("Monterey County's zip code scraper is missing rows")

In [28]:
try:
    assert not len(export_df) > 33
except AssertionError:
    raise AssertionError("Monterey County's zip code scraper has more rows than before")

## Export

Set date

In [29]:
tz = pytz.timezone("America/Los_Angeles")

In [30]:
today = datetime.now(tz).date()

In [31]:
slug = "monterey"

In [32]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [33]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [34]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [35]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [36]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)